In [ ]:
# === Imports ===
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# === LEITURA E PRÉ-PROCESSAMENTO ===
df = pd.read_csv("dados_rios_ficticios.csv", encoding="utf-8", sep=";")
df.columns = [col.encode('utf-8').decode('utf-8-sig').strip().lower() for col in df.columns]

for col in df.columns:
    if "data" in col:
        df.rename(columns={col: "data"}, inplace=True)
        break

df["data"] = pd.to_datetime(df["data"], format="%d/%m/%Y", errors="coerce")

def classificar_nivel(nivel):
    if nivel < 5.5:
        return "Normal"
    elif 5.5 <= nivel < 7.5:
        return "Atenção"
    else:
        return "Risco"

df["nivel_categoria"] = df["nivel_metro"].apply(classificar_nivel)
df["dia"] = df["data"].dt.day
df["mes"] = df["data"].dt.month

le = LabelEncoder()
df["rio_codificado"] = le.fit_transform(df["rio"])

# === TREINAMENTO ===
X = df[["chuva_mm", "vazao_m3s", "dia", "mes", "rio_codificado"]]
y = df["nivel_categoria"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)

# === INTERFACE SIMPLES ===
mapa_rios = dict(zip(le.classes_, le.transform(le.classes_)))

print("🔍 Previsão de Risco do Nível do Rio")
print("Rios disponíveis:", ", ".join(mapa_rios.keys()))

rio = input("Digite o nome do rio: ")
while rio not in mapa_rios:
    rio = input("Nome inválido. Digite novamente: ")

print("\n🌧️ Sobre a chuva recente na sua região:")
print("1. Não choveu")
print("2. Chuva fraca")
print("3. Chuva média")
print("4. Chuva forte")

chuva_opcao = input("Selecione o nível de chuva (1 a 4): ")
chuva_map = {"1": 0.0, "2": 5.0, "3": 20.0, "4": 50.0}
while chuva_opcao not in chuva_map:
    chuva_opcao = input("Entrada inválida. Escolha de 1 a 4: ")

chuva = chuva_map[chuva_opcao]

data_input = input("Digite a data da observação (formato DD/MM/AAAA): ")
data_formatada = pd.to_datetime(data_input, format="%d/%m/%Y", errors="coerce")
if pd.isna(data_formatada):
    print("❌ Data inválida. Tente novamente.")
    exit()

dia = data_formatada.day
mes = data_formatada.month

df_filtrado = df[(df["rio"] == rio) & (df["mes"] == mes)]
if df_filtrado.empty:
    print("❌ Sem dados históricos para esse rio nesse mês.")
    exit()

vazao_media = df_filtrado["vazao_m3s"].mean()

entrada = pd.DataFrame([[chuva, vazao_media, dia, mes, mapa_rios[rio]]],
                       columns=["chuva_mm", "vazao_m3s", "dia", "mes", "rio_codificado"])

risco = modelo.predict(entrada)[0]
print(f"\n📢 Resultado: O nível do rio está em estado de **{risco.upper()}**.")
